In [1]:
#used to import data
import pandas as pd
#Evaluates the model
from sklearn.metrics import log_loss
import time #just used to see how long the model takes to run
from pytorch_tabnet.tab_model import TabNetRegressor

from sklearn.model_selection import train_test_split

In [2]:
df_11 = pd.read_csv('data/pbp/ready_to_model/version_4/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/version_4/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/version_4/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/version_4/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/version_4/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/version_4/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/version_4/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/version_4/17_18_pbp_cleaned.csv')
df_19 = pd.read_csv('data/pbp/ready_to_model/version_4/18_19_pbp_cleaned.csv')
df_20 = pd.read_csv('data/pbp/ready_to_model/version_4/19_20_pbp_cleaned.csv')
df_21 = pd.read_csv('data/pbp/ready_to_model/version_4/20_21_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17])

In [3]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop(['is_goal'], axis=1).to_numpy()
    y = df.is_goal.to_numpy()
    
    return X,y

X,y = get_X_y(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=101)

#X_train.values
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [ ]:
reg = TabNetRegressor(device_name)
model = reg.fit(X_train, y_train, 
                eval_set = [(X_train, y_train), (X_val, y_val)],
                eval_name=['train','val'],
                eval_metric = ['logloss'],
                max_epochs = 20)

In [ ]:
preds = reg.predict(X_test)
print(log_loss(y_test, preds))

In [ ]:
#after 10 epochs logloss = .197
#after 10 epochs logloss = .197
#Added a validation set, 10 epochs logloss = .198
#With the validation set and 10 epochs, logloss = .1965 <--- slight improvement
#increased the number of steps to 5, logloss = .200
#increased the number of steps to 8, logloss = .207
#increased the number of steps to 2, logloss = .1976
#using 3 as the default fir number of steps is fine 
#Setting the n_independent to 4 results in heavy overfitting
#set n_shared to 4 and logloss increased
#Setting labmda_sparse to 2.5e-4 was slightly better, logloss = .1965
#using entmax instead of sparsemax logloss = .1976
#Increasing the batch size did have a high logloss = .1985
#however, it was still falling, I will repeat with 20 epochs and compare
#increasing the epochs did little for the performance
#Increasing the num_workers to 2 gave a logloss of .1972 but was still falling, will allow more epochs
#After 25 epochs I got logloss = .1966
#in the end the defaults worked the best. 

In [4]:
reg = TabNetRegressor(device_name = 'cuda')
model = reg.fit(X_train, y_train, 
                eval_set = [(X_train, y_train), (X_val, y_val)],
                eval_name=['train','val'],
                eval_metric = ['logloss'],
                max_epochs = 20)

Device used : cuda
epoch 0  | loss: 0.06197 | train_logloss: 0.2015  | val_logloss: 0.20493 |  0:00:27s
epoch 1  | loss: 0.05133 | train_logloss: 0.21584 | val_logloss: 0.22183 |  0:00:54s
epoch 2  | loss: 0.05078 | train_logloss: 0.19615 | val_logloss: 0.20041 |  0:01:20s
epoch 3  | loss: 0.05064 | train_logloss: 0.19663 | val_logloss: 0.20151 |  0:01:45s
epoch 4  | loss: 0.05034 | train_logloss: 0.19357 | val_logloss: 0.19883 |  0:02:11s
epoch 5  | loss: 0.05015 | train_logloss: 0.19361 | val_logloss: 0.19894 |  0:02:40s
epoch 6  | loss: 0.05032 | train_logloss: 0.19539 | val_logloss: 0.20005 |  0:03:11s
epoch 7  | loss: 0.05032 | train_logloss: 0.19536 | val_logloss: 0.20092 |  0:03:40s
epoch 8  | loss: 0.05025 | train_logloss: 0.19671 | val_logloss: 0.20102 |  0:04:40s
epoch 9  | loss: 0.05015 | train_logloss: 0.19947 | val_logloss: 0.20483 |  0:05:10s
epoch 10 | loss: 0.05005 | train_logloss: 0.19304 | val_logloss: 0.19753 |  0:05:35s
epoch 11 | loss: 0.0499  | train_logloss: 0.19